# 2-Level Full Factorial
# Generating Data

# Preliminaries
## Importing packages

In [1]:
import numpy as np
import pandas as pd
import itertools as it

import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

from thesis_EAfunc import *
from thesis_visfunc import *

In [2]:
import datetime
time_a = datetime.datetime.now()
print(time_a)

2019-05-20 11:30:09.624554


# Define Landscape

In [3]:
#Problem domain
x_min = -15
x_max = 15
y_min = -15
y_max = 15

#Known minimum
x_point = -1
y_point = -1

domain = (x_min, x_max, y_min, y_max)
point = (x_point, y_point)
img_size = (8.5, 4.25)

#Problem definition
def f(x, y):
    D = 2
    alpha = 1/8
    
    x=(x-5)/6
    y=(y-5)/6
    
    a = np.abs(x ** 2 + y ** 2 - D) ** (alpha * D)
    b = ( 0.5 * (x ** 2 + y ** 2) + (x + y) ) / D
        
    return (a + b + 0.5)

#Testing the minimum
f(-1,-1)

0.0

# Running the Evolutionary Algorithm
Experiments -> Each trial with a different mutation probability and mutation size -> Each trial with 50 runs
## Setting the EA's parameters

In [4]:
#starting seed
np.random.seed(42)

## Setting data storage

In [5]:
mult_fit_cols = ['exp'] + ['pop_s'] + ['b'] + ['mut_p'] + ['mut_s'] + ['p_sel'] + ['s_sel'] + ['run', 'generation', 'fitness_min', 'fitness_max', 'fitness_mean', 'fitness_std']
multi_fit = pd.DataFrame(columns=mult_fit_cols)
multi_fit = multi_fit.infer_objects()

multi_gen_cols = ['exp'] + ['pop_s'] + ['b'] + ['mut_p'] + ['mut_s'] + ['p_sel'] + ['s_sel'] + ['run', 'birthdate', 'generation', 'function', 'fitness', 'gen_x', 'gen_y']
multi_gen = pd.DataFrame(columns=multi_gen_cols)
multi_gen = multi_gen.infer_objects()

## Parameters common to all experiments

In [6]:
#Algorithm parameters
## Number of experiments, and generations per experiment
run_n = 30
gen_f = 60

## Population size
pop_s = [10, 30]

## Parent subpopulation's selection method and size
par_selection = ['fitness_proportional_selection','tournament_k3']
b = [0.5, 5]
par_s = [z*y for z in pop_s for y in b]

## Progeny subpopulation's and size
prog_s = par_s
### Crossover Method
crossover = 'uniform'
### Mutation method, probability and size
mutation = 'random_all_gau_dis'
mut_p = [0.1, 0.9]
mut_s = [.1, 10]

## New population selection method
sur_selection = ['fitness_proportional_selection','tournament_k3']

## Iterative experiment

In [7]:
print()
exp_par = list(it.product(pop_s, b, mut_p, mut_s, par_selection, sur_selection))
len(exp_par)

64

In [16]:
%%time
exp_n = 1
for (zz, yy, xx, vv, uu, tt) in exp_par:
    sur_selection = tt
    par_selection = uu
    mut_s = vv
    mut_p = xx
    b = yy
    pop_s = zz
    prog_s = int(b * pop_s)
    par_s = prog_s
    
    genera_res, fitness_res = EA_exp(run_n, gen_f, f, domain, pop_s, par_s, prog_s, mut_p, mut_s, par_selection, crossover, mutation, sur_selection)
    
    fitness_res.insert(0, 's_sel', tt)
    fitness_res.insert(0, 'p_sel', uu)
    fitness_res.insert(0, 'mut_s', vv)
    fitness_res.insert(0, 'mut_p', xx)
    fitness_res.insert(0, 'b', yy)
    fitness_res.insert(0, 'pop_s', zz)
    fitness_res.insert(0, 'exp', exp_n)
    multi_fit = multi_fit.append(fitness_res, ignore_index=True, sort=False)
    multi_fit = multi_fit.infer_objects()
    
    genera_res.insert(0, 's_sel', tt)
    genera_res.insert(0, 'p_sel', uu)
    genera_res.insert(0, 'mut_s', vv)
    genera_res.insert(0, 'mut_p', xx)
    genera_res.insert(0, 'b', yy)
    genera_res.insert(0, 'pop_s', zz)
    genera_res.insert(0, 'exp', exp_n)
    multi_gen = multi_gen.append(genera_res, ignore_index=True, sort=False)
    multi_gen = multi_gen.infer_objects()
        
    exp_n += 1

Wall time: 1h 41min 16s


In [17]:
multi_fit.to_pickle('./EA_A_05_fit.gz', compression='gzip')
multi_gen.to_pickle('./EA_A_05_gen.gz', compression='gzip')

In [8]:
multi_fit = pd.read_pickle('./EA_A_05_fit.gz', compression='gzip')
multi_gen = pd.read_pickle('./EA_A_05_gen.gz', compression='gzip')

In [9]:
time_b = datetime.datetime.now()
print(time_b)
print(time_b-time_a)

2019-05-20 11:30:25.668126
0:00:16.043572


# Data summary for Analysis

In [20]:
['pop_s'] + ['b'] + ['mut_p'] + ['mut_s'] + ['p_sel'] + ['s_sel']
multi_fit.tail()

,exp,pop_s,b,mut_p,mut_s,p_sel,s_sel,run,generation,fitness_min,fitness_max,fitness_mean,fitness_std
117115,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,56.0,0.485122,3.004269,1.054857,0.531125
117116,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,57.0,0.485122,2.004721,1.164721,0.407348
117117,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,58.0,0.485122,2.918143,1.237652,0.554010
117118,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,59.0,0.485122,2.725057,1.393787,0.587630
117119,64,30,5.0,0.9,10.0,tournament_k3,tournament_k3,29,60.0,0.440289,2.814151,1.274528,0.634896


In [48]:
query = (multi_fit['generation']==gen_f)
multi_final_fitness_res = multi_fit[query]
multi_final_fitness_res = multi_final_fitness_res.groupby(['pop_s'] + ['b'] + ['mut_p'] + ['mut_s'] + ['p_sel'] + ['s_sel']).mean().add_suffix('_MEAN').reset_index()
multi_final_fitness_res.drop(['exp_MEAN', 'run_MEAN', 'generation_MEAN'], axis=1, inplace=True)
multi_final_fitness_res.columns = ['pop_s', 'b', 'mut_p', 'mut_s', 'p_sel', 's_sel', 'f_min_MEAN', 'f_max_MEAN', 'f_mean_MEAN', 'f_std_MEAN']
multi_final_fitness_res['pop_s'] = multi_final_fitness_res['pop_s'].replace([10, 30], [-1, 1]).infer_objects()
multi_final_fitness_res['b'] = multi_final_fitness_res['b'].replace([.5, 5], [-1, 1]).infer_objects()
multi_final_fitness_res['mut_p'] = multi_final_fitness_res['mut_p'].replace([.1, .9], [-1, 1]).infer_objects()
multi_final_fitness_res['mut_s'] = multi_final_fitness_res['mut_s'].replace([.1, 10], [-1, 1]).infer_objects()
multi_final_fitness_res['p_sel'] = multi_final_fitness_res['p_sel'].replace(['fitness_proportional_selection','tournament_k3'], [-1, 1]).infer_objects()
multi_final_fitness_res['s_sel'] = multi_final_fitness_res['s_sel'].replace(['fitness_proportional_selection','tournament_k3'], [-1, 1]).infer_objects()
multi_final_fitness_res

,pop_s,b,mut_p,mut_s,p_sel,s_sel,f_min_MEAN,f_max_MEAN,f_mean_MEAN,f_std_MEAN
0,-1,-1.0,-1.0,-1.0,-1,-1,0.770816,0.779056,0.775203,0.003117
1,-1,-1.0,-1.0,-1.0,-1,1,0.901820,0.916325,0.912887,0.005835
2,-1,-1.0,-1.0,-1.0,1,-1,0.694523,0.709522,0.703910,0.006153
3,-1,-1.0,-1.0,-1.0,1,1,0.676936,0.689657,0.681660,0.004524
4,-1,-1.0,-1.0,1.0,-1,-1,0.757939,0.757939,0.757939,0.000000
5,-1,-1.0,-1.0,1.0,-1,1,0.795490,0.795490,0.795490,0.000000
6,-1,-1.0,-1.0,1.0,1,-1,0.750336,0.750336,0.750336,0.000000
7,-1,-1.0,-1.0,1.0,1,1,0.783611,0.783611,0.783611,0.000000
8,-1,-1.0,1.0,-1.0,-1,-1,0.449589,0.505973,0.460654,0.019835
9,-1,-1.0,1.0,-1.0,-1,1,0.358110,0.431312,0.375206,0.024959


In [49]:
multi_final_fitness_res.to_pickle('./EA_A_05_data.gz', compression='gzip')

In [ ]:
sns.lmplot(x='mut_p', y='fitness_min', order=3, data=multi_final_fitness_res,hue='mut_s', col='mut_s', x_bins=5, col_wrap=3, x_ci='sd')

In [ ]:
sns.lmplot(x='mut_s', y='fitness_min', order=3, data=multi_final_fitness_res,hue='mut_p', x_bins=5)

In [ ]:
sns.lmplot(x='mut_p', y='fitness_min', order=3, data=multi_final_fitness_res,hue='mut_s', x_bins=5)

In [ ]:
time_c = datetime.datetime.now()
print(time_c)
print(time_c-time_a)

In [ ]:
multi_fit.info()
print()
multi_gen.info()

In [ ]:
multi_final_fitness_res.info()